In [150]:
import server
import requests
import os
import random

In [78]:
os.environ['FLASK_CONFIG'] = './local/server-config-dev.yaml'
app = server.create_app()

In [84]:
models = server.models
db = server.models.db

In [172]:
with app.app_context():
    q = db.session.query(models.Tag)
    q = q.filter_by(has_explicit=True)
    tags = q.all()

tag_names = sorted([t.tag for t in tags])

In [173]:
tag_names

['apps',
 'aws',
 'containers',
 'docker',
 'healthtech',
 'hippa',
 'jill',
 'tutorial']

In [176]:
def k_unique(vals, k=3):
    output = set()
    while len(output) < k:
        output.add(random.choice(vals))
    return list(output)

In [571]:
search_tags = k_unique(tag_names, k=2)

with app.app_context():
    q = db.session.query(models.Post, db.func.count(models.Tag.tag))
    q = q.select_from(models.Post)
    q = q.join(models.Post.post_tags)
    q = q.join(models.PostTag.tag)
    q = q.filter(models.Tag.tag.in_(search_tags))
    q = q.group_by(models.Post)
    q = q.having(db.func.count(models.Tag.tag) >= len(search_tags))
    
    results = q.all()
    output = []
    for r in results:
        post, count = r
        post_tags = [p.tag.tag.lower() for p in post.post_tags]
        post_tags.sort()
        output.append((post, post_tags))

In [572]:
print('search:')
print('\t{}'.format(search_tags))
print('results:')
for post, post_tags in output:
    print('\tpost: {}'.format(post.id), '-> tags:',post_tags)

search:
	['containers', 'tutorial']
results:
	post: 5 -> tags: ['4', 'containers', 'healthtech', 'post', 'tutorial']
	post: 9 -> tags: ['8', 'apps', 'containers', 'post', 'tutorial']
